# 1. Import Required Libraries

Import libraries for data manipulation, file operations, and JSON parsing.

In [3]:
import pandas as pd
import os
import json
import glob

# Directory containing JSON result files
RESULTS_DIR = os.path.join('data', 'results')

# 2. Load JSON Result Files

Load all backtest JSON result files into Python structures.

In [4]:
results_list = []
for filepath in glob.glob(os.path.join(RESULTS_DIR, '*.json')):
    with open(filepath, 'r') as f:
        data = json.load(f)
        results_list.append(data)

len(results_list)  # Number of files loaded

42

# 3. Aggregate Results into a DataFrame

Combine the loaded JSON results into a pandas DataFrame for analysis.

In [7]:
records = []
for res in results_list:
    # Each JSON file contains a single strategy-market result
    records.append({
        'strategy': res.get('strategy_name'),
        'market_type': res.get('market_type'),
        'total_return': res.get('total_return'),
        'win_rate': res.get('win_rate'),
        'sharpe_ratio': res.get('sharpe_ratio'),
        'max_drawdown_pct': res.get('max_drawdown_pct', None)
    })

df = pd.DataFrame(records)
df.head()

,strategy,market_type,total_return,win_rate,sharpe_ratio,max_drawdown_pct
0,TrendFollowing,sideways market,0.00,0.0,0.00,None
1,GridTrading,normal market,1.47,1.0,0.38,None
2,GridTrading,missing market,0.39,1.0,0.00,None
3,DCA,sideways market,0.01,1.0,0.01,None
4,TrendFollowing,missing market,0.00,0.0,0.00,None


# 4. Generate Summary Report

Create pivot tables for key metrics across strategies and market types.

In [8]:
metrics = ['total_return', 'win_rate', 'sharpe_ratio', 'max_drawdown_pct']
pivots = {}
for metric in metrics:
    pivots[metric] = df.pivot(index='strategy', columns='market_type', values=metric)
    display(pivots[metric])

market_type,gappy market,invalid market,missing market,normal market,sideways market,trending market,volatile market
strategy,,,,,,,
Arbitrage,-3.24,-0.23,3.30,18.34,0.32,20.30,-0.68
DCA,-0.30,0.57,0.95,-0.13,0.01,0.34,0.19
GridTrading,-0.03,-0.30,0.39,1.47,0.05,2.23,-0.27
MeanReversion,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Scalping,15.75,0.35,2.97,-14.58,-11.16,14.70,2.93
TrendFollowing,0.00,0.00,0.00,0.00,0.00,0.00,0.00


market_type,gappy market,invalid market,missing market,normal market,sideways market,trending market,volatile market
strategy,,,,,,,
Arbitrage,0.0000,0.0000,1.0,1.0,1.0,1.00,0.0000
DCA,0.3333,0.9630,1.0,0.5,1.0,1.00,0.8667
GridTrading,0.0000,0.0000,1.0,1.0,1.0,1.00,0.0000
MeanReversion,0.0000,0.0000,0.0,0.0,0.0,0.00,0.0000
Scalping,1.0000,0.6667,0.5,0.0,0.0,0.75,0.5000
TrendFollowing,0.0000,0.0000,0.0,0.0,0.0,0.00,0.0000


market_type,gappy market,invalid market,missing market,normal market,sideways market,trending market,volatile market
strategy,,,,,,,
Arbitrage,-0.39,-0.01,0.0,0.54,0.04,2.01,0.01
DCA,-0.39,0.71,0.0,-0.23,0.01,0.97,0.09
GridTrading,-0.03,-0.41,0.0,0.38,0.03,1.95,-0.13
MeanReversion,0.00,0.00,0.0,0.00,0.00,0.00,0.00
Scalping,0.69,0.04,0.0,-2.86,-0.41,1.51,0.16
TrendFollowing,0.00,0.00,0.0,0.00,0.00,0.00,0.00


market_type,gappy market,invalid market,missing market,normal market,sideways market,trending market,volatile market
strategy,,,,,,,
Arbitrage,None,None,None,None,None,None,None
DCA,None,None,None,None,None,None,None
GridTrading,None,None,None,None,None,None,None
MeanReversion,None,None,None,None,None,None,None
Scalping,None,None,None,None,None,None,None
TrendFollowing,None,None,None,None,None,None,None


## 4.b Compute Dynamic Weight Table

Based on the Sharpe ratio pivot, calculate strategy weights per market by normalizing performance scores, and export to a JSON file for use in the EnsembleStrategy.

In [ ]:
# Normalize Sharpe ratios to get weights per market type
sharpe_df = pivots['sharpe_ratio'].fillna(0)
# Convert negative Sharpe to zero to avoid negative weights
sharpe_df[sharpe_df < 0] = 0
# Normalize columns so weights sum to 1 per market
weight_df = sharpe_df.div(sharpe_df.sum(axis=0), axis=1).fillna(0)

# Convert to nested dict and write to JSON
weights = weight_df.to_dict()
with open('data/weight_table.json', 'w') as f:
    json.dump(weights, f, indent=2)

weights

## 4.a Visualization: Heatmaps and Bar Charts
Use Plotly Express to visualize key metrics across strategies and market types.

In [9]:
import plotly.express as px

In [10]:
for metric, pivot in pivots.items():
    fig = px.imshow(
        pivot,
        text_auto=True,
        aspect='auto',
        color_continuous_scale='Viridis',
        labels=dict(x="Market Type", y="Strategy", color=metric.replace('_', ' ').title()),
        x=pivot.columns,
        y=pivot.index,
        title=f"{metric.replace('_', ' ').title()} Heatmap"
    )
    fig.show()

In [11]:
for metric, pivot in pivots.items():
    df_long = pivot.reset_index().melt(id_vars='strategy', var_name='market_type', value_name=metric)
    fig = px.bar(
        df_long,
        x='strategy',
        y=metric,
        color='market_type',
        barmode='group',
        title=f"{metric.replace('_', ' ').title()} by Strategy and Market",
        labels={"strategy": "Strategy", metric: metric.replace('_', ' ').title(), "market_type": "Market Type"}
    )
    fig.show()

/Users/antenehtessema/Desktop/botsalgo/venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/Users/antenehtessema/Desktop/botsalgo/venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/Users/antenehtessema/Desktop/botsalgo/venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



/Users/antenehtessema/Desktop/botsalgo/venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



# 5. Export Summary Report

Save the aggregated DataFrame and pivot tables to CSV and generate a Markdown version.

In [17]:
# Ensure summaries directory exists and export CSV
os.makedirs('data', exist_ok=True)
df.to_csv('data/summary_report.csv', index=False)

# Markdown version of total_return pivot
try:
    import tabulate  # ensure dependency
    total_return_md = pivots['total_return'].to_markdown()
    print(total_return_md)
except Exception:
    print("Could not generate markdown table. Displaying raw DataFrame:")
    display(pivots['total_return'])

| strategy       |   gappy market |   invalid market |   missing market |   normal market |   sideways market |   trending market |   volatile market |
|:---------------|---------------:|-----------------:|-----------------:|----------------:|------------------:|------------------:|------------------:|
| Arbitrage      |          -3.24 |            -0.23 |             3.3  |           18.34 |              0.32 |             20.3  |             -0.68 |
| DCA            |          -0.3  |             0.57 |             0.95 |           -0.13 |              0.01 |              0.34 |              0.19 |
| GridTrading    |          -0.03 |            -0.3  |             0.39 |            1.47 |              0.05 |              2.23 |             -0.27 |
| MeanReversion  |           0    |             0    |             0    |            0    |              0    |              0    |              0    |
| Scalping       |          15.75 |             0.35 |             2.97 |          -14.5

# 6. Optional: Integration with Web Dashboard or CI

- **Web Dashboard**: In `web_dashboard/src/`, create a component to fetch and display the generated `data/summary_report.csv` or embed the interactive Plotly charts.
- **CI Pipeline**: In your CI (e.g., GitHub Actions), add a step to run this notebook headlessly (using `nbconvert --execute`) and commit `data/summary_report.csv` and any updated visual outputs on each push.